In [49]:
import numpy as np

In [50]:
timetable_constraints = {
  "nbr_of_days": 5,
  "nbr_of_periods_per_day": 4,
  "nbr_of_subjects": 5,
  "nbr_of_groups": 5,
  "periods_by_subject": [4, 4, 4, 2, 2],
  "subject_names": ["Maths", "Français", "Anglais", "SVT", "PC"],
  "group_names": ["Groupe 1", "Groupe 2", "Gr 3", "Gr 4", "Gr 5"],
  "preferences":{
    "total_count": 8,
    "morning_classes": [0, 3, 4],
  }
}

In [51]:
class Timetable:
  def __init__(self, npArray, constraints):
    self.constraints = constraints
    # timetable as a 3d array ( group, day, period) => subject from npArray
    self.timetable = npArray.reshape(
      (constraints["nbr_of_groups"], constraints["nbr_of_days"], constraints["nbr_of_periods_per_day"])
    )
    pass

  def toNpArray(self):
    return self.timetable.flatten()

  def isValid(self):
    pass

  def fitness(self):
    # return 1 if it is a valid timetable
    # return 0 if it is totally not
    # return a value between 0 and 0.8 if it is partially valid
    # a valid timetable is a timetable that respects all the constraints
    # a partially valid timetable is a timetable that respects some of the constraints
    # a totally invalid timetable is a timetable that respects none of the constraints
    # the value returned is the percentage of constraints that are respected

    respected_constraints = 0
    total_constraints = 0

    for group in self.timetable:
      subject_periods_in_table = np.zeros(self.constraints["nbr_of_subjects"])
      for day in group :
        for period in day:
          if period == -1: continue
          subject_periods_in_table[period]+=1
      respected_constraints += np.sum(subject_periods_in_table == self.constraints["periods_by_subject"])
      total_constraints += self.constraints["nbr_of_subjects"]
      
    score = 0.6*respected_constraints/total_constraints if respected_constraints < total_constraints else 0.8

    bonus = 0
    # check if the morning classes constraint is respected
    for group in self.timetable:
      for day in group:
        for period in day:
          if period == -1: continue
          if period in self.constraints["preferences"]["morning_classes"] and (day[0] == period or day[1] == period):
            bonus += 1
            break

    score += 0.2*bonus / self.constraints["preferences"]["total_count"]

    return score + 0.0000000000001

  def plot(self):
    pass

  def print(self):
    print(self.fitness())
    i=0
    for group in self.timetable:
      i+=1
      print("Group : "+str(i))
      print(group)

In [52]:
import pygad

In [53]:
def fitness_func(ga_instance, solution, solution_idx):
    timeTable = Timetable(solution, timetable_constraints)
    fitness = timeTable.fitness()
    return fitness

In [54]:
fitness_function = fitness_func

num_generations = 1000
num_parents_mating = 10

sol_per_pop = 100
gene_space = [ 
              { 
                "low" : -1, 
                "high": timetable_constraints["nbr_of_subjects"],
                "step": 1
              } 
              for _ in range(timetable_constraints["nbr_of_groups"] * timetable_constraints["nbr_of_days"] * timetable_constraints["nbr_of_periods_per_day"])
             ]
num_genes = timetable_constraints["nbr_of_days"] * timetable_constraints["nbr_of_periods_per_day"] * timetable_constraints["nbr_of_groups"]
gene_type = [int for _ in range(num_genes)]

parent_selection_type = "sus"
keep_parents = 1

crossover_type = "single_point"

mutation_type = "random"
mutation_percent_genes = 10

In [56]:
from tqdm import tqdm
progress_bar = tqdm(total=num_generations, bar_format='{l_bar}{bar:20}{r_bar}{bar:-10b}')

def on_gen(ga_instance):
    progress_bar.update(1)
    progress_bar.set_description_str("Fitness={fitness:.4f}\t".format(fitness=ga_instance.best_solution()[1]))
    # print("Fitness of the best solution :", ga_instance.best_solution()[1])

ga_instance = pygad.GA(num_generations=num_generations,
                       on_generation=on_gen,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       gene_space=gene_space,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                      #  mutation_probability=[0.5, 0.1],
                       gene_type=gene_type,
                       num_genes=num_genes
                      )

ga_instance.run()
ga_instance.plot_fitness()



Fitness=0.9360000000001002:  27%|█████▍              | 273/1000 [00:27<01:13,  9.86it/s]






































KeyboardInterrupt: 

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))

Parameters of the best solution : [4 3 3 0 1 3 2 1 2 -1 -1 0 -1 0 2 0 3 2 3 -1 0 1 1 0 -1 0 3 0 4 1 0 1 -1
 -1 3 -1 4 0 -1 2 -1 1 2 3 2 2 4 1 -1 0 0 3 2 4 0 0 4 -1 4 4 1 0 4 0 3 -1
 4 0 3 -1 2 -1 2 0 2 2 -1 1 1 1 2 0 2 1 3 2 2 -1 1 0 1 3 1 -1 1 0 0 4 1 4]
Fitness value of the best solution = 1.0080000000001


In [ ]:
timetable = Timetable(solution, timetable_constraints)
timetable.print()

1.0080000000001
Group : 1
[[4 3 3 0]
 [1 3 2 1]
 [2 -1 -1 0]
 [-1 0 2 0]
 [3 2 3 -1]]
Group : 2
[[0 1 1 0]
 [-1 0 3 0]
 [4 1 0 1]
 [-1 -1 3 -1]
 [4 0 -1 2]]
Group : 3
[[-1 1 2 3]
 [2 2 4 1]
 [-1 0 0 3]
 [2 4 0 0]
 [4 -1 4 4]]
Group : 4
[[1 0 4 0]
 [3 -1 4 0]
 [3 -1 2 -1]
 [2 0 2 2]
 [-1 1 1 1]]
Group : 5
[[2 0 2 1]
 [3 2 2 -1]
 [1 0 1 3]
 [1 -1 1 0]
 [0 4 1 4]]
